MUST CHANGE I/P DIRRECTORY AND O/P DIRECTORY PATH DEPENDING UPON WHICH FILE IS BEING PROCESSED. 
THERE ARE TWO TYPES OF I/P FILES THAT ARE USING THE CODE FOR GMM PREDICTION.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
#!pip install https://github.com/ipython-contrib/jupyter_contrib_nbextensions/tarball/master
#!pip install jupyter_contrib_nbextensions
#!jupyter contrib nbextension install --user
#!jupyter nbextension enable hinterland/hinterland:


In [ ]:
#!jupyter nbextension enable hinterland/hinterland

In [ ]:
%cd /content/drive/My Drive/MSCodeGoogleColab/gmr

/content/drive/My Drive/MSCodeGoogleColab/gmr


In [ ]:
import pandas as pd
import pandas as pd
from pandas import read_csv
import numpy as np
from datetime import datetime

##### Global Constants

indextoprocess = 0          # 0 = systolic | 1 = diastolic | 2 = heart rate
basedir = '/content/drive/My Drive/MSCodeGoogleColab'
my_list = list()
meanvalue = 0.0

if indextoprocess == 0: 
  meanvalue = 120.0
if indextoprocess == 1: 
  meanvalue = 80.0
if indextoprocess == 2:
  meanvalue = 60.0
  
#opfilepath = ''

predarr = []

In [ ]:
def getuserinputfromfile():
  #filename = basedir+'/ajantip.csv'
  
  filename = basedir+'/ajantip_results/complete_timeseries/testing/testing_data_125miss.csv'
  ##  set op file path
  opfilepath = basedir+'/ajantip_results/complete_timeseries/testing/testing_result_125miss.csv'
  # '/content/drive/My Drive/MSCodeGoogleColab/ajantip_results/' + str(indextoprocess)+"_second_proprocessed_result.csv"
  data = pd.read_csv(filename, header=None)
  #datacopy = data.copy(deep=True)
  #bloodpressure = data.iloc[:, idxx]
  #bloodpressure = pd.np.array(bloodpressure).astype('float')
  print('opfilepath:: '+opfilepath)
  return data, opfilepath
  

In [ ]:
def getoutlierinputfromfile(fileidx):
  filename = basedir+'/ajantip_results/'+ str(indextoprocess)+'_time_bins/outlier_results/mod'+str(fileidx)+'.csv'
  print('reading outliers filled file at:: ', str(filename))
  ##  set op file path
  opfilepath = '/content/drive/My Drive/MSCodeGoogleColab/ajantip_results/' + str(indextoprocess)+'_complete_timebins/'
  print(opfilepath)
 
  data = pd.read_csv(filename, header=None, skiprows=1)
  datacopy = data.copy(deep=True)
  bloodpressure = data.iloc[:, 0]
  bloodpressure = pd.np.array(bloodpressure).astype('float')
  print('items read from file ', len(bloodpressure))
  return bloodpressure, opfilepath

In [ ]:
#import pandas as pd
#import pandas as pd
#from pandas import read_csv
#import numpy as np
#from datetime import datetime
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error

from gmm import GMM


#basedir = '/content/drive/My Drive/MSCodeGoogleColab'
#filename = basedir+'/ajantip.csv'
#data = pd.read_csv(filename, header=None)

#my_list = list()
#datacopy = data.copy(deep=True)
#bloodpressure = data.iloc[:, indextoprocess]
#bloodpressure = pd.np.array(bloodpressure).astype('float')

#normalized_time = pd.to_datetime(data.iloc[:, 3])
#normalized_time = pd.np.array(normalized_time).astype('float')
previousIndex = 0
n = 0


In [ ]:
def processData(currentWindow):
        print('currentwindow: ', currentWindow)
        gmm1 = GMM(n_components=1,random_state=0)
     # train with window and predict  the immediate next value
        trainingData = currentWindow       
#         print(trainingData) 
        totalData = np.ndarray((trainingData.size, 2))
#         print(totalData)
        for l in range(0, trainingData.size-1):
          totalData[l, 0] = trainingData[l]
          totalData[l, 1] = trainingData[l + 1]
          
        totalData[trainingData.size-1,0] = trainingData[trainingData.size-1]
        totalData[trainingData.size - 1, 1] = 0 
        trainDDD = totalData[0:-1,:]

        if (len(trainDDD) == 0):
          print('trainDDD size 0 - check')
          print(totalData)
        try:
          np.asarray_chkfinite(trainDDD)
        except ValueError:
          print('ValueError')
          print('nan in trainDDD::', np.isnan(trainDDD).any())
        return gmm1,totalData  
   

In [ ]:
def predictNextZeroValueIfExists(T ,i,Y):
        trainDDD = totalData[0:-1,:]
        if len(bloodpressure) > i+1:
          TArray2 = np.ndarray((1, 2))
          print("Test Index Before "+str(i))
          for k in range(i+1, bloodpressure.size):
            if bloodpressure[k] == 0:
              print("Test Index Loop "+str(k))
              #trains the gmm with predicted value
              gmm2 = GMM(n_components=1,random_state=0)
              TArray2[0,0] = T
              TArray2[0,1] = Y
              trainDDD = np.vstack((trainDDD,TArray2))
                        
              try:
                np.asarray_chkfinite(trainDDD)
              except ValueError:
                print('Data ValueError    Please Check sundus ')
                print(TArray2)
                print(np.isnan(trainDDD).any())
              print("second window")  
              print(trainDDD)             
              gmm2.from_samples(trainDDD)
              TArray1 = np.ndarray((1, 1))
              TArray1[0, 0] = int(Y)

              Yy = gmm2.predict(np.array([0]), TArray1)
              if (math.isnan(Yy)):
                print('Predicted value for Yy is nan**')
                print('TArray1::', TArray1)
                Yy = meanvalue
                
              Yy =  math.ceil(Yy)
              if T-Yy > 30:
                     Yy = T
              print("Other Predicted Value "+str(Yy))
              
              bloodpressure[k] = Yy
              print("output added: "+str(k)+" : "+str(Yy))
              predarr.append((k, Yy))  
              #predarr.append((k+1, Y))  
              i = k
            else:
              i = k -1
              break;
        return i     

In [ ]:
bloodpressure, opfilepath = getuserinputfromfile()
#getoutlierinputfromfile(0)
print('bp::', bloodpressure)
print(predarr)

In [ ]:
data, opfilepath = getuserinputfromfile()

sys = data.iloc[:, 0]
dias = data.iloc[:, 1]
hr = data.iloc[:, 2] 
predsys = [0] * len(sys)
preddias = [0] * len(sys)
predhr = [0] * len(sys)

rres = {'PredSys': predsys, 'PredDias': preddias,'PredHR': predhr,}
result = pd.DataFrame(rres)
 
print(result.iloc[:,0])

In [ ]:
# from IPython.core.debugger import set_trace
previousIndex = 0
import math

data, opfilepath = getuserinputfromfile()

sys = data.iloc[:, 0]
dias = data.iloc[:, 1]
hr = data.iloc[:, 2]

predsys = [0] * len(sys)
preddias = [0] * len(sys)
predhr = [0] * len(sys)

rres = {'PredSys': predsys, 'PredDias': preddias,'PredHR': predhr,}
result = pd.DataFrame(rres)

for r in range(0,3):  
  bloodpressure = data.iloc[:, r]      #getoutlierinputfromfile(r)
  bloodpressure = pd.np.array(bloodpressure).astype('float')
  predarr = []
  previousIndex = 0
  i = 0 

  while i < bloodpressure.size:
    print("current idx: "+str(i))
    if (bloodpressure[i]!=0):
      print("output added: "+str(i)+" : "+str(bloodpressure[i]))
      predarr.append((i, bloodpressure[i]))

    if(bloodpressure[i]==0):
      print("zero detected at index "+str(i))

      if(bloodpressure[i] == 0):
        #if (i-previousIndex) < 15:
           #previousIndex = previousIndex -(15-(i-previousIndex))
          
        if (i-previousIndex) > 15:
          print("updating previndex value from: "+str(previousIndex)+" to: "+str(i - 15))
          previousIndex = i - 15
          
        print("reads the blood pressure value from previous index "+ str(previousIndex)+":"+str(i))
        print(bloodpressure[previousIndex:i])
        print("**********************************")
        currentWindow = bloodpressure[previousIndex:i]
        print("items in currwindow: "+str(currentWindow.shape[0]))
        if currentWindow.shape[0] > 0:
          print("current window size is greater than zero")
          gmm,totalData = processData(currentWindow)
          gmm.from_samples(np.array(totalData))
          T = totalData[-1,0]
          TArray = np.ndarray((1,1))
          TArray[0,0] = T
          print("****************** To Predict****************")
          print(np.array([0])+ TArray)
          print("**********************************")
          Y = gmm.predict(np.array([0]), TArray)

          print("First Predicted Value: "+str(Y)+" at idx:"+str(i))        
          if (math.isnan(Y)):
            print('Predicted value is nan-- or 0')
            Y = meanvalue
            
          if Y == 0:
            Y = meanvalue

          Y = math.ceil(Y)
          if T-Y > 30:
            Y = T
          valtoadd = int(Y)
          print("this output added: "+str(i)+" : "+str(Y))
          predarr.append((i, valtoadd))
          print("Existing Index "+str(i))
          bloodpressure[i] = valtoadd # sets the zaro 
          # i is index Y is previous predicted Value and T is data
          i = predictNextZeroValueIfExists(T ,i,Y) 
          
          print("updating previndex after prediction: "+str(previousIndex)+" to: "+str(i))
          previousIndex = i
    i = i+1
  
  import csv
  print('*********************************')
  print('predictions::')
  print(len(predarr))
  print('*********************************')

  dtype = [('Col1','int32'), ('Col2','float32')]
  index = [i for i in range(0, len(predarr))]
  df = pd.DataFrame(predarr, index=index)

  if r == 0:
    result['PredSys'] = df.iloc[:,1] #pd.DataFrame({'PredSys': predarr})
  if r == 1:
    result['PredDias'] = df.iloc[:,1] #pd.DataFrame({'PredDias': predarr})
  if r == 2:
    result['PredHR'] = df.iloc[:,1] #pd.DataFrame({'PredHR': predarr})

  name= str(indextoprocess)+"_faasecond_proprocessed_result.csv"

  #cff = opfilepath + str(r) + '_complete.csv'
  #print('opfilepath:' + opfilepath)
  #export_csv = result.to_csv(opfilepath, index = None, header = True)
  

print('opfilepath:' + opfilepath)
export_csv = result.to_csv(opfilepath, index = None, header = True)
  

In [ ]:
  import csv
  print('*********************************')
  print('predictions::')
#  print(predarr)
  print('*********************************')
  
  
  dtype = [('Col1','int32'), ('Col2','float32')]
  values = predarr
  index = ['Row'+str(i) for i in range(1, len(values)+1)]

  df = pd.DataFrame(values, index=index)
  
  #print(df)
  name= str(indextoprocess)+"_aasecond_proprocessed_result.csv"
  
  cff = opfilepath + str(r) + '_complete.csv'
  print('opfilepath:' + cff)
  export_csv = df.to_csv(cff, index = None, header = True)
  
  #export_csv = df.to_csv ('/content/drive/My Drive/MSCodeGoogleColab/ajantip_results/'+name, index = None, header=True)




In [ ]:
import numpy as np

def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [ ]:
from sklearn.metrics.regression import mean_absolute_error as mae
from sklearn.metrics.regression import mean_squared_error as mse

filename = basedir+'/ajantip_results/complete_timeseries/testing/combined_results/testing_result_82miss.csv'
data = pd.read_csv(filename)
print(data[0:10])

j=0
while j<5:
  print('j: ',j)
  X = data.iloc[:,j]
  res = data.iloc[:,j+1]
  j = j+2
  meanerror = mae(X.values, res.values)
  meansquareerror = mse(X.values, res.values)
  rmse_val = math.sqrt(mse(X.values, res.values))
  rms = np.sqrt(meansquareerror)
  mape = mean_absolute_percentage_error(X.values, res.values)

  print('mean error: ', meanerror)
  print('mean square error: ', meansquareerror)
  print('root mean square error: ', rms)
  print('mape: ', mape)



In [ ]:
meanerror = mae(X.values, res.values)
meansquareerror = mse(X.values, res.values)
rmse_val = math.sqrt(mse(X.values, res.values))
rms = np.sqrt(meansquareerror)

print('type: ' + str(type))
print(meansquareerror)
print(rms)
print(meanerror)